In [2]:
include("JuliRay.jl")
Base.@irrational ° 0.0174532925199432957692369076848861271344 (big(pi)/big(180))

In [105]:
ε=10^(-3)

function f(x)
    ifelse(x>0,exp(-1/x),0)
end
function g(x)
    f(2x/√3)/(f(2x/√3)+f(2/√3-2x/√3))
end
function Smooth(a,b,x)
    g((x-a)/(b-a))
end

VERTEX=Int
EDGE=Array{VERTEX,1}
FACE=Array{EDGE,1}
CELL=Array{FACE,1}

function vertices(edge::EDGE)
    return edge
end
function vertices(face::FACE)
    f=copy(face)
    v=union(vcat(f...))
    w=[v[1]]
    for j ∈ 1:(length(v)-1)
        i=findfirst(e->w[end] ∈ e, f)
        push!(w,filter(v->v≠w[end],f[i])[1])
        deleteat!(f,i)
    end
    return w
end
function vertices(cell::CELL)
    return DeleteDuplicates(vcat(vcat(cell...)...))
end
function edges(face::FACE)
    return face
end
function edges(cell::CELL)
    return DeleteDuplicates(sort.(vcat(cell...)))
end
function faces(cell::CELL)
    return cell
end

import Base.∈
function ∈(vertex::VERTEX, face::FACE)
    return vertex ∈ vertices(face)
end
function ∈(vertex::VERTEX, cell::CELL)
    return vertex ∈ vertices(cell)
end
function ∈(edge::EDGE, cell::CELL)
    return edge ∈ edges(cell)
end

function DualRegularPlolyhedron(POINTS,c)
    fs=copy(c)
    es=edges(fs)
    vs=vertices(fs)
    POINTS_=normalize.([+(POINTS[vertices(f)]...) for f ∈ fs])
    vs_=collect(1:length(faces(fs)))
    es_=[findall(f->e ∈ f, fs) for e ∈ es]
    fs_=[es_[findall(e->issubset(e,findall(f->v ∈ f, c)),es_)] for v ∈ vs]
    return POINTS_, fs_
end

function ℝ³⭢S³(p::Array{T,1}) where T<:Real
    if(length(p)≠3)
        error("No Point on ℝ³")
    end
    return [2p[1]/(1+p[1]^2+p[2]^2+p[3]^2),2p[2]/(1+p[1]^2+p[2]^2+p[3]^2),2p[3]/(1+p[1]^2+p[2]^2+p[3]^2),(-1+p[1]^2+p[2]^2+p[3]^2)/(1+p[1]^2+p[2]^2+p[3]^2)]
end
function ℝ³⭢S³(p::Array{T,1},θ::Real) where T<:Real
    if(length(p)≠3)
        error("No Point on ℝ³")
    elseif(θ<ε)
        return [p[1:3]...,0.0]
    else
        c=[0,0,0,cot(θ)]
        r=1/sin(θ)
        P=p*cot((π/2-θ)/2+π/4)
        return ℝ³⭢S³(P)*r+c
    end
end
function S³⭢ℝ³(q::Array{T,1}) where T<:Real
    if(length(q)≠4)
        error("No Point on S³")
    elseif(!(norm(q)≈1.0))
        print(norm(q))
        error("No Point on unit S³, radius: $(norm(q))")
    end
    return [q[1]/(1-q[4]),q[2]/(1-q[4]),q[3]/(1-q[4])]
end
function S³⭢ℝ³(q::Array{T,1},θ::Real) where T<:Real
    if(length(q)≠4)
        error("No Point on S³")
    elseif(θ<ε)
        return q[1:3]
    else
        c=[0,0,0,cot(θ)]
        r=1/sin(θ)
        return S³⭢ℝ³((q-c)/r)*tan((π/2-θ)/2+π/4)
    end
end
function ℝ⁴⭢S³(p::Array{T,1},θ::Real) where T<:Real
    if(θ<ε)
        return [p[1:3]...,0.0]
    else
        R=1/sin(θ)
        O=[0,0,0,cot(θ)]
        return O+R*normalize(p-O)
    end
end

function NormalVector(p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector)
    A=hcat(p₁-p₄,p₂-p₄,p₃-p₄)
    return normalize([(-1)^i*det(A[deleteat!(collect(1:4),i),:]) for i ∈ 1:4])
end

function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector)
    𝒏=NormalVector(p₁,p₂,p₃)
    return q-2*dot(𝒏,q-p₁)*𝒏
end
function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector)
    𝒏=NormalVector(p₁,p₂,p₃,p₄)
    return q-2*dot(𝒏,q-p₁)*𝒏
end
function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector, θ)
    𝒏=NormalVector(p₁,p₂,p₃,p₄)
    return ℝ⁴⭢S³(q-2*dot(𝒏,q-p₁)*𝒏,θ)
end

function PickFace(cell::CELL, v::RealVector,POINTS³)
    if(norm(v)≈0)
        error("vector v must be non-zero")
    end
    faces=vertices.(cell)
    fpts=(i->POINTS³[i]).(faces)
    return cell[findmax([dot(v,+(pts...)) for pts ∈ fpts])[2]]
end

function NewCell(cell::CELL,v::RealVector,θ,POINTS³, POINTS⁴)
    n=length(POINTS³)
    c=copy(cell)
    face=PickFace(cell,v,POINTS³)
    IND_face=vertices(face)
    IND_cell=Int[]
    for f ∈ c for e ∈ f for v ∈ e push!(IND_cell,v) end end end
    IND_cell=union(IND_cell)
    IND_cell2=[
        if(i ∈ IND_face)
            i
        else
            push!(POINTS³,Mirror(POINTS³[i],POINTS³[IND_face[1]],POINTS³[IND_face[2]],POINTS³[IND_face[3]]));
            if(θ<ε)
                push!(POINTS⁴,[POINTS³[end]...,0]);
            else
                O=[0,0,0,cot(θ)]
                push!(POINTS⁴,Mirror(POINTS⁴[i],O,POINTS⁴[IND_face[1]],POINTS⁴[IND_face[2]],POINTS⁴[IND_face[3]],θ));
            end
            n=n+1
        end
        for i ∈ IND_cell]
    return [[[IND_cell2[findfirst(w->w==v,IND_cell)] for v ∈ e] for e ∈ f] for f ∈ c]
end

function SphericalSphere(v,r::Real,θ::Real) where T<:RealVector
    V=S³⭢ℝ³(v,θ)
    return Sphere(V,r)
end

function SphericalCylinder(v₁,v₂,r::Real,θ::Real) where T<:RealVector
    w₁=ℝ⁴⭢S³((v₁+v₂)/2,θ)
    V₁=S³⭢ℝ³(v₁,θ)
    V₂=S³⭢ℝ³(v₂,θ)
    W₁=S³⭢ℝ³(w₁,θ)
    return Arc(V₁,W₁,V₂,r)
end

function SphericalPolygon(v::Array{T,1},θ::Real) where T<:RealVector
    n=length(v)
    u=ℝ⁴⭢S³(+(v...)/n,θ)
    v₁=v[1]
    v₂=v[2]
    v₃=v[3]
    V=(q->S³⭢ℝ³(q,θ)).(v)
    U=S³⭢ℝ³(u,θ)
    w=[ℝ⁴⭢S³((v[i]+v[mod(i,length(v))+1])/2,θ) for i ∈ 1:length(v)]
    W=(q->S³⭢ℝ³(q,θ)).(w)
    if(θ<ε)
        return Polygon(V)
    elseif(rank(hcat(V...),atol=1.0e-12)==2)
        m=4
        vw=copy(v)
        for _ ∈ 1:m
            l=length(vw)
            vw=[ℝ⁴⭢S³((vw[(i+1)÷2]+vw[mod(i÷2,l)+1])/2,θ) for i ∈ 1:2l]
        end
        VW=(q->S³⭢ℝ³(q,θ)).(vw)
        return csgUnion(Polygon(VW))
    else
        O=Circumcenter(U,V[1],V[2],V[3])
        sphere=Sphere(O,norm(U-O))
        N=NormalVector(V[1],W[1],V[2]);
        direction=sign(dot(N,O-U))

        cylinders=csgIntersection([
                (V₁=V[i];
                V₂=V[mod(i,n)+1];
                V₃=V[mod(i+1,n)+1];
                C=Circumcenter(V₁,W[i],V₂);
                N=NormalVector(V₁,W[i],V₂);
                direction=sign(dot(N,U-W[i]));
                cylinder=Cylinder(C,C+2*direction*norm(U-O)*N,norm(U-O)))
                for i ∈ 1:n
                ])
        return csgClip(sphere,cylinders)
    end
end

function Cells2Object(cells::Array{CELL,1},θ,POINTS⁴;rᵥ=0.05,rₑ=0.025,color=RGB(1,1,1))
    cs=copy(cells)
    fs=DeleteDuplicates(vcat(cs...))
    es=DeleteDuplicates(vcat(fs...))
    vs=DeleteDuplicates(vcat(es...))
    V=rgbColor(csgUnion([SphericalSphere(POINTS⁴[v],rᵥ,θ) for v ∈ vs]),RGB(0.1,0.1,0.1))
    E=rgbColor(csgUnion([SphericalCylinder(POINTS⁴[e[1]],POINTS⁴[e[2]],rₑ,θ) for e ∈ es]),RGB(0.2,0.2,0.2))
    F=rgbftColor(csgUnion([SphericalPolygon([POINTS⁴[i] for i ∈ vertices(f)],θ) for f ∈ fs]),color,FT(0.1,0.3))
    return csgUnion(V,E,F)
end

Cells2Object (generic function with 1 method)

In [106]:
# F₄
𝒑₁=[1,1,1]
𝒑₂=[1,-1,-1]
𝒑₃=[-1,1,-1]
𝒑₄=[-1,-1,1]
POINTS=[𝒑₁,𝒑₂,𝒑₃,𝒑₄]
v₁=1
v₂=2
v₃=3
v₄=4
e₁=[v₁,v₂]
e₂=[v₁,v₃]
e₃=[v₁,v₄]
e₄=[v₂,v₃]
e₅=[v₂,v₄]
e₆=[v₃,v₄]

f₁=[e₄,e₅,e₆]
f₂=[e₂,e₃,e₆]
f₃=[e₁,e₃,e₅]
f₄=[e₁,e₂,e₄]
c₁=[f₁,f₂,f₃,f₄]
const F₄=normalize.(POINTS), c₁

# F₆
𝒑₁=[1,1,1]
𝒑₂=[1,1,-1]
𝒑₃=[1,-1,1]
𝒑₄=[1,-1,-1]
𝒑₅=[-1,1,1]
𝒑₆=[-1,1,-1]
𝒑₇=[-1,-1,1]
𝒑₈=[-1,-1,-1]
POINTS=[𝒑₁,𝒑₂,𝒑₃,𝒑₄,𝒑₅,𝒑₆,𝒑₇,𝒑₈]
v₁=1
v₂=2
v₃=3
v₄=4
v₅=5
v₆=6
v₇=7
v₈=8
e₁=[v₁,v₂]
e₂=[v₁,v₃]
e₃=[v₁,v₅]
e₄=[v₂,v₄]
e₅=[v₂,v₆]
e₆=[v₃,v₄]
e₇=[v₃,v₇]
e₈=[v₄,v₈]
e₉=[v₅,v₆]
e₁₀=[v₅,v₇]
e₁₁=[v₆,v₈]
e₁₂=[v₇,v₈]
f₁=[e₁,e₂,e₄,e₆]
f₂=[e₁,e₃,e₅,e₉]
f₃=[e₂,e₃,e₇,e₁₀]
f₄=[e₄,e₅,e₈,e₁₁]
f₅=[e₆,e₇,e₈,e₁₂]
f₆=[e₉,e₁₀,e₁₁,e₁₂]
c₁=[f₁,f₂,f₃,f₄,f₅,f₆]
const F₆=normalize.(POINTS), c₁

const F₈=DualRegularPlolyhedron(F₆...)

(Array{Float64,1}[[0.5773502691896258, 0.5773502691896258, 0.5773502691896258], [0.5773502691896258, 0.5773502691896258, -0.5773502691896258], [0.5773502691896258, -0.5773502691896258, 0.5773502691896258], [0.5773502691896258, -0.5773502691896258, -0.5773502691896258], [-0.5773502691896258, 0.5773502691896258, 0.5773502691896258], [-0.5773502691896258, 0.5773502691896258, -0.5773502691896258], [-0.5773502691896258, -0.5773502691896258, 0.5773502691896258], [-0.5773502691896258, -0.5773502691896258, -0.5773502691896258]], Array{Array{Int64,1},1}[[[1, 2], [1, 3], [2, 4], [3, 4]], [[1, 2], [1, 5], [2, 6], [5, 6]], [[1, 3], [1, 5], [3, 7], [5, 7]], [[2, 4], [2, 6], [4, 8], [6, 8]], [[3, 4], [3, 7], [4, 8], [7, 8]], [[5, 6], [5, 7], [6, 8], [7, 8]]])

In [5]:
function C₈(θ)
    h=1/2
    r=√(1-h^2)

    R=1/sin(θ)
    O=[0,0,0,cot(θ)]
    N=O+[0,0,0,R]
    H=O-[0,0,0,√(R^2-r^2)]

    POINTS, c₁=F₆
    POINTS=r*copy(POINTS)
    POINTS³=copy(POINTS)
    POINTS⁴=[[𝒑...,0]+H for 𝒑 ∈ POINTS]
    c₂=NewCell(c₁,[-1,0,0],θ,POINTS³, POINTS⁴)
    c₃=NewCell(c₂,[-1,0,0],θ,POINTS³, POINTS⁴)
    c₄=NewCell(c₁,[1,0,0],θ,POINTS³, POINTS⁴)
    c₅=NewCell(c₁,[0,0,-1],θ,POINTS³, POINTS⁴)
    c₆=NewCell(c₅,[0,-1,0],θ,POINTS³, POINTS⁴)
    c₇=NewCell(c₆,[0,-1,0],θ,POINTS³, POINTS⁴)
    c₈=NewCell(c₅,[0,1,0],θ,POINTS³, POINTS⁴)
    
    cells=[c₁,c₂,c₃,c₄,c₅,c₆,c₇,c₈]
    return cells, POINTS³, POINTS⁴
end

C₈ (generic function with 1 method)

In [6]:
M=120
for i ∈ 0:2M
    θ=π/2*(Smooth(0,1,i/M)-Smooth(1,2,i/M))
    cells, POINTS³, POINTS⁴=C₈(θ)
    render(Cells2Object(cells,θ,POINTS⁴,color=RGB(0.2,1,1),rₑ=0.012,rᵥ=0.03),camera=LngLatCamera(lng=180°+360°*i/M,lat=25°,pers=0.2,zoom=0.15,width=1200,height=900),name="8-Cell",index=i+1)
end

Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /buildworker/worker/package_linux64/build/src/locks.h:166 [inlined]
jl_typeinf_end at /buildworker/worker/package_linux64/build/src/gf.c:2595
typeinf_ext at ./compiler/typeinfer.jl:569
typeinf_ext at ./compiler/typeinfer.jl:599
jfptr_typeinf_ext_1.clone_1 at /home/horikawa/julia/julia-1.2.0/lib/julia/sys.so (unknown line)
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2191
jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1614 [inlined]
jl_type_infer at /buildworker/worker/package_linux64/build/src/gf.c:207
jl_compile_method_internal at /buildworker/worker/package_linux64/build/src/gf.c:1773
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2196
top-level scope at ./In[6]:5
jl_toplevel_eval_flex at /buildworker/worker/package_linux64/build/src/toplevel.c:809
jl_toplevel_eval_in at /buildworker/worker/package_linux64/build/src/topl

Rendered 1080000 of 1080000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 1200 x 900
----------------------------------------------------------------------------
Pixels:          1149000   Samples:          182628   Smpls/Pxl: 0.16
Rays:            1916822   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Cone/Cylinder                   354036          140066     39.56
Polygon                        3008892         1212246     40.29
Sphere                           31502           11700     37.14
Bounding Box                  56314828        13559124     24.08
----------------------------------------------------------------------------
Shadow Ray Tests:           1347528   Succeeded:             

Rendered 1080000 of 1080000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 1200 x 900
----------------------------------------------------------------------------
Pixels:          1149000   Samples:          195264   Smpls/Pxl: 0.17
Rays:            1965745   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Cone/Cylinder                   378172          145329     38.43
Polygon                        3227063         1303424     40.39
Sphere                           51997           12858     24.73
Bounding Box                  60905351        14619560     24.00
----------------------------------------------------------------------------
Shadow Ray Tests:           1444136   Succeeded:             

POV-Ray finished

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 9.1.0 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Di

POV-Ray finished

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 9.1.0 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Di

...

Rendered 1080000 of 1080000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 1200 x 900
----------------------------------------------------------------------------
Pixels:          1149000   Samples:          207171   Smpls/Pxl: 0.18
Rays:            2046764   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Cone/Cylinder                   454793          155799     34.26
Polygon                        3648667         1480171     40.57
Sphere                           55958           13152     23.50
Bounding Box                  68107707        16476029     24.19
----------------------------------------------------------------------------
Shadow Ray Tests:           1627073   Succeeded:             

Rendered 1080000 of 1080000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 1200 x 900
----------------------------------------------------------------------------
Pixels:          1149000   Samples:          202365   Smpls/Pxl: 0.18
Rays:            2051377   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Cone/Cylinder                   468699          149612     31.92
Polygon                        3726351         1514433     40.64
Sphere                           48947           13245     27.06
Bounding Box                  69468075        16808450     24.20
----------------------------------------------------------------------------
Shadow Ray Tests:           1657469   Succeeded:             


povray: received signal SIGINT: Interrupt; requested render cancel
File '8-Cell_000008.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '8-Cell_000008.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '8-Cell_000008.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '8-Cell_000008.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '8-Cell_000008.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '8-Cell_000008.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '8-Cell_000008.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '8-Cell_000008.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '8-Cell_000008.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '8-Cell_000008.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '8-Cell_000008.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '8-Cell_000008.pov' line 6: Parse Warning: Poly

Rendered 674304 of 1080000 pixels (62%)
Fatal error in renderer: Frontend halted render.
Render cancelled by user



ProcessFailedException: failed process: Process(`povray 8-Cell_000008.pov`, ProcessExited(2)) [2]


In [7]:
function C₁₆(θ)
    h=1/2
    r=√(1-h^2)

    R=1/sin(θ)
    O=[0,0,0,cot(θ)]
    N=O+[0,0,0,R]
    H=O-[0,0,0,√(R^2-r^2)]

    POINTS, c₁=F₄
    POINTS=r*copy(POINTS)
    POINTS³=copy(POINTS)
    POINTS⁴=[[𝒑...,0]+H for 𝒑 ∈ POINTS]
    V=[1,-2,0]
    c₂=NewCell(c₁,V,θ,POINTS³, POINTS⁴)
    c₃=NewCell(c₂,V,θ,POINTS³, POINTS⁴)
    c₄=NewCell(c₃,V,θ,POINTS³, POINTS⁴)
    c₅=NewCell(c₁,-V,θ,POINTS³, POINTS⁴)
    c₆=NewCell(c₅,-V,θ,POINTS³, POINTS⁴)
    c₇=NewCell(c₆,-V,θ,POINTS³, POINTS⁴)
    c₈=NewCell(c₇,-V,θ,POINTS³, POINTS⁴)
    c₉=NewCell(c₁,[-2,-1,0],θ,POINTS³, POINTS⁴)
    W=[-1,-0.5,1]
    c₁₀=NewCell(c₉,W,θ,POINTS³, POINTS⁴)
    c₁₁=NewCell(c₁₀,W,θ,POINTS³, POINTS⁴)
    c₁₂=NewCell(c₁₁,W,θ,POINTS³, POINTS⁴)
    c₁₃=NewCell(c₉,-W,θ,POINTS³, POINTS⁴)
    c₁₄=NewCell(c₁₃,-W,θ,POINTS³, POINTS⁴)
    c₁₅=NewCell(c₁₄,-W,θ,POINTS³, POINTS⁴)
    c₁₆=NewCell(c₁₅,-W,θ,POINTS³, POINTS⁴)
    
    cells=[c₁,c₂,c₃,c₄,c₅,c₆,c₇,c₈,c₉,c₁₀,c₁₁,c₁₂,c₁₃,c₁₄,c₁₅,c₁₆]
    return cells, POINTS³, POINTS⁴
end

C₁₆ (generic function with 1 method)

In [81]:
M=120
for i ∈ 0:2M
    θ=π/2*(Smooth(0,1,i/M)-Smooth(1,2,i/M))
    cells, POINTS³, POINTS⁴=C₁₆(θ)
    render(Cells2Object(cells,θ,POINTS⁴,color=RGB(0.2,1,1)),camera=LngLatCamera(lng=180°+360°*i/M,lat=25°,pers=0.2,zoom=0.12,width=600,height=450),name="16-Cell",index=i+1)
end

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Parser Options
  Input file: 16-Cell_000002.pov
  Remove bounds........On 
  Split unions.........Off
  Library paths:
  Clock value:    0.000  (Animation off)
Image Output Options
  Image resolution.....600 by 450 (rows 1 to 450, columns 1 to 600).
  Output file..........16-Cell_000002.png, 24 bpp PNG
  Dithering............Off
  Graphic display......Off
  Mosaic preview.......Off
  Continued trace......Off
Information Output Options
  All Streams to console..........On 
  Debug Stream to console.........On 
  Fatal Stream to console.........On 
  Render Stream to console........On 
  Statistics Stream to console....On 
  Warning Stream to console.......On 
==== [Parsing...] ==========================================================
File '16-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell

File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' li

File '16-Cell_000004.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000004.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000004.pov' line 6: Parse Warning: Polygon not closed. Closing it.
----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:          119
Infinite Objects:          0
Light Sources:             1
Total:                   120
----------------------------------------------------------------------------
Parser Time
  Parse Time:       0 hours  0 minutes  0 seconds (0.002 seconds)
              using 1 thread(s) with 0.002 CPU-seconds total
  Bounding Time:    0 hours  0 minutes  0 seconds (0.000 seconds)
              using 1 thread(s) with 0.000 CPU-seconds total
----------------------------------------------------------------------------
Render Options
  Quality:  9
  Bounding boxes....

----------------------------------------------------------------------------
Render Options
  Quality:  9
  Bounding boxes.......On   Bounding threshold: 3
  Antialiasing.........On  (Method 1, Threshold 0.300, Depth 3, Jitter 1.00,
 Gamma 2.50)
==== [Rendering...] ========================================================
Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:           60615   Smpls/Pxl: 0.21
Rays:             469337   Saved:              2192   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Cone/Cylinder                  1236298           55359      4.48
Polygon      

...

Parser Options
  Input file: 16-Cell_000020.pov
  Remove bounds........On 
  Split unions.........Off
  Library paths:
  Clock value:    0.000  (Animation off)
Image Output Options
  Image resolution.....600 by 450 (rows 1 to 450, columns 1 to 600).
  Output file..........16-Cell_000020.png, 24 bpp PNG
  Dithering............Off
  Graphic display......Off
  Mosaic preview.......Off
  Continued trace......Off
Information Output Options
  All Streams to console..........On 
  Debug Stream to console.........On 
  Fatal Stream to console.........On 
  Render Stream to console........On 
  Statistics Stream to console....On 
  Warning Stream to console.......On 
==== [Parsing...] ==========================================================
----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:           47
Infinite Objects:         72
Light Sources:            

Parser Options
  Input file: 16-Cell_000022.pov
  Remove bounds........On 
  Split unions.........Off
  Library paths:
  Clock value:    0.000  (Animation off)
Image Output Options
  Image resolution.....600 by 450 (rows 1 to 450, columns 1 to 600).
  Output file..........16-Cell_000022.png, 24 bpp PNG
  Dithering............Off
  Graphic display......Off
  Mosaic preview.......Off
  Continued trace......Off
Information Output Options
  All Streams to console..........On 
  Debug Stream to console.........On 
  Fatal Stream to console.........On 
  Render Stream to console........On 
  Statistics Stream to console....On 
  Warning Stream to console.......On 
==== [Parsing...] ==========================================================
----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:           69
Infinite Objects:         50
Light Sources:            

----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:          104
Infinite Objects:         15
Light Sources:             1
Total:                   120
----------------------------------------------------------------------------
Parser Time
  Parse Time:       0 hours  0 minutes  0 seconds (0.003 seconds)
              using 1 thread(s) with 0.003 CPU-seconds total
  Bounding Time:    0 hours  0 minutes  0 seconds (0.000 seconds)
              using 1 thread(s) with 0.000 CPU-seconds total
----------------------------------------------------------------------------
Render Options
  Quality:  9
  Bounding boxes.......On   Bounding threshold: 3
  Antialiasing.........On  (Method 1, Threshold 0.300, Depth 3, Jitter 1.00,
 Gamma 2.50)
==== [Rendering...] ========================================================
Rendered 270000 of 270000 pixels (100%)
------

Parser Options
  Input file: 16-Cell_000026.pov
  Remove bounds........On 
  Split unions.........Off
  Library paths:
  Clock value:    0.000  (Animation off)
Image Output Options
  Image resolution.....600 by 450 (rows 1 to 450, columns 1 to 600).
  Output file..........16-Cell_000026.png, 24 bpp PNG
  Dithering............Off
  Graphic display......Off
  Mosaic preview.......Off
  Continued trace......Off
Information Output Options
  All Streams to console..........On 
  Debug Stream to console.........On 
  Fatal Stream to console.........On 
  Render Stream to console........On 
  Statistics Stream to console....On 
  Warning Stream to console.......On 
==== [Parsing...] ==========================================================
----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:          109
Infinite Objects:         10
Light Sources:            

----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:          109
Infinite Objects:         10
Light Sources:             1
Total:                   120
----------------------------------------------------------------------------
Parser Time
  Parse Time:       0 hours  0 minutes  0 seconds (0.005 seconds)
              using 1 thread(s) with 0.004 CPU-seconds total
  Bounding Time:    0 hours  0 minutes  0 seconds (0.000 seconds)
              using 1 thread(s) with 0.000 CPU-seconds total
----------------------------------------------------------------------------
Render Options
  Quality:  9
  Bounding boxes.......On   Bounding threshold: 3
  Antialiasing.........On  (Method 1, Threshold 0.300, Depth 3, Jitter 1.00,
 Gamma 2.50)
==== [Rendering...] ========================================================
Rendered 270000 of 270000 pixels (100%)
------

In [208]:
function C₂₄(θ)
    h=1/√2
    r=√(1-h^2)

    R=1/sin(θ)
    O=[0,0,0,cot(θ)]
    N=O+[0,0,0,R]
    H=O-[0,0,0,√(R^2-r^2)]

    POINTS, c₁=F₈
    POINTS=r*copy(POINTS)
    POINTS³=copy(POINTS)
    POINTS⁴=[[𝒑...,0]+H for 𝒑 ∈ POINTS]
    c₂=NewCell(c₁,[1,1,1],θ,POINTS³, POINTS⁴)
    c₃=NewCell(c₂,[1,1,1],θ,POINTS³, POINTS⁴)
    c₄=NewCell(c₃,[1,1,1],θ,POINTS³, POINTS⁴)
    c₅=NewCell(c₁,-[1,1,1],θ,POINTS³, POINTS⁴)
    c₆=NewCell(c₅,-[1,1,1],θ,POINTS³, POINTS⁴)
    c₇=NewCell(c₁,[1,-1,-1],θ,POINTS³, POINTS⁴)
    c₈=NewCell(c₁,[-1,1,-1],θ,POINTS³, POINTS⁴)
    c₉=NewCell(c₁,[-1,-1,1],θ,POINTS³, POINTS⁴)
    c₁₀=NewCell(c₂,[-1,0,0],θ,POINTS³, POINTS⁴)
    c₁₁=NewCell(c₂,[0,-1,0],θ,POINTS³, POINTS⁴)
    c₁₂=NewCell(c₂,[0,0,-1],θ,POINTS³, POINTS⁴)
    c₁₃=NewCell(c₃,[1,-1,-1],θ,POINTS³, POINTS⁴)
    c₁₄=NewCell(c₃,[-1,1,-1],θ,POINTS³, POINTS⁴)
    c₁₅=NewCell(c₃,[-1,-1,1],θ,POINTS³, POINTS⁴)
    c₁₆=NewCell(c₄,[-1,0,0],θ,POINTS³, POINTS⁴)
    c₁₇=NewCell(c₄,[0,-1,0],θ,POINTS³, POINTS⁴)
    c₁₈=NewCell(c₄,[0,0,-1],θ,POINTS³, POINTS⁴)
    c₁₉=NewCell(c₆,[1,-1,-1],θ,POINTS³, POINTS⁴)
    c₂₀=NewCell(c₆,[-1,1,-1],θ,POINTS³, POINTS⁴)
    c₂₁=NewCell(c₆,[-1,-1,1],θ,POINTS³, POINTS⁴)
    c₂₂=NewCell(c₅,[-1,0,0],θ,POINTS³, POINTS⁴)
    c₂₃=NewCell(c₅,[0,-1,0],θ,POINTS³, POINTS⁴)
    c₂₄=NewCell(c₅,[0,0,-1],θ,POINTS³, POINTS⁴)

    
    cells=[c₁,c₂,c₃,c₄,c₅,c₆,c₇,c₈,c₉,c₁₀,c₁₁,c₁₂,c₁₃,c₁₄,c₁₅,c₁₆,c₁₇,c₁₈,c₁₉,c₂₀,c₂₁,c₂₂,c₂₃,c₂₄]
    return cells, POINTS³, POINTS⁴
end

C₂₄ (generic function with 1 method)

In [209]:
i=0
θ=0
cells, POINTS³, POINTS⁴=C₂₄(θ)
render(Cells2Object(cells,θ,POINTS⁴,color=RGB(0.2,1,1)),camera=LngLatCamera(lng=180°+360°*i/M,lat=25°,pers=0.2,zoom=0.12,width=600,height=450),name="24-Cell")

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 9.1.0 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell.pov' line 6: Parse Warni

Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:          151110   Smpls/Pxl: 0.53
Rays:             691696   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Cone/Cylinder                  2485560          144505      5.81
Polygon                        4538293          770264     16.97
Sphere                           30471           11202     36.76
Bounding Box                  56266397        17752992     31.55
----------------------------------------------------------------------------
Shadow Ray Tests:            704909   Succeeded:                

Process(`povray 24-Cell.pov`, ProcessExited(0))

In [210]:
M=120
for i ∈ 0:2M
    θ=π/2*(Smooth(0,1,i/M)-Smooth(1,2,i/M))
    cells, POINTS³, POINTS⁴=C₂₄(θ)
    render(Cells2Object(cells,θ,POINTS⁴,color=RGB(0.2,1,1)),camera=LngLatCamera(lng=180°+360°*i/M,lat=25°,pers=0.2,zoom=0.12,width=600,height=450),name="24-Cell",index=i+1)
end

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 9.1.0 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000001.pov' li

Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:          151110   Smpls/Pxl: 0.53
Rays:             691696   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Cone/Cylinder                  2482692          144498      5.82
Polygon                        4538293          770264     16.97
Sphere                           30471           11202     36.76
Bounding Box                  56266397        17752992     31.55
----------------------------------------------------------------------------
Shadow Ray Tests:            704909   Succeeded:                

File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '24-Cell_000002.pov' li

Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:          146511   Smpls/Pxl: 0.51
Rays:             685371   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Cone/Cylinder                  2537111          140521      5.54
Polygon                        4589256          767846     16.73
Sphere                           31029           11135     35.89
Bounding Box                  56755947        18006101     31.73
----------------------------------------------------------------------------
Shadow Ray Tests:            706412   Succeeded:                

...

POV-Ray finished

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 9.1.0 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Di

Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:          146916   Smpls/Pxl: 0.51
Rays:             680783   Saved:               300   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Box                           28695446        10845821     37.80
Cone/Cylinder                  2735262          133359      4.88
CSG Intersection              28695446         2192386      7.64
Sphere                       233128170       211612736     90.77
Torus                         14347723           46064      0.32
Torus Bound                   14347723           61990      0.43
Clipping Ob

Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:          144495   Smpls/Pxl: 0.50
Rays:             675946   Saved:               326   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Box                          213199020        75932653     35.62
Cone/Cylinder                  1269490           58117      4.58
CSG Intersection             213199020        14886081      6.98
Sphere                       231184082       210051835     90.86
Torus                        106599510          591337      0.55
Torus Bound                  106599510          828066      0.78
Clipping Ob

Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:          142425   Smpls/Pxl: 0.50
Rays:             671731   Saved:               265   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Box                          283448350       101233391     35.71
Cone/Cylinder                   418838           11975      2.86
CSG Intersection             283448350        20177696      7.12
Sphere                       229672905       207644304     90.41
Torus                        141724175          833757      0.59
Torus Bound                  141724175         1190616      0.84
Clipping Ob

Excessive output truncated after 524415 bytes.----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:          342
Infinite Objects:        121
Light Sources:             1
Total:                   464
----------------------------------------------------------------------------
Parser Time
  Parse Time:       0 hours  0 minutes  0 seconds (0.011 seconds)
              using 1 thread(s) with 0.010 CPU-seconds total
  Bounding Time:    0 hours  0 minutes  0 seconds (0.000 seconds)
              using 1 thread(s) with 0.000 CPU-seconds total
----------------------------------------------------------------------------
Render Options
  Quality:  9
  Bounding boxes.......On   Bounding threshold: 3
  Antialiasing.........On  (Method 1, Threshold 0.300, Depth 3, Jitter 1.00,
 Gamma 2.50)
==== [Rendering...] ========================================================
